<a href="https://colab.research.google.com/github/9219193025/smart-qna-app/blob/master/Smart_Resume_Q%26A_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain-community faiss-cpu sentence-transformers transformers accelerate pymupdf gradio duckduckgo-search


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/12

In [ ]:
!pip install langchain langchain-community faiss-cpu sentence-transformers transformers accelerate pymupdf gradio duckduckgo-search
from google.colab import files
uploaded = files.upload()

file_paths = list(uploaded.keys())  # list of uploaded files
!pip install -U langchain-community
!pip install pymupdf

from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("Kartikey Tiwari resume  4.pdf")
docs = loader.load()
print(f"✅ Loaded pages: {len(docs)}")

from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = splitter.split_documents(docs)
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = FAISS.from_documents(chunks, embedding_model)
retriever = db.as_retriever(search_kwargs={"k": 3})
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

from transformers import pipeline
from langchain.llms.base import LLM
from pydantic import Extra, Field
from typing import Optional, List, Any

class LocalLLM(LLM):
    pipeline: Any = Field(default=None)
    class Config:
        extra = Extra.allow

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.pipeline = pipeline("text2text-generation", model="google/flan-t5-large")

    @property
    def _llm_type(self) -> str:
        return "flan_t5"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        return self.pipeline(prompt[:1500], max_new_tokens=256)[0]["generated_text"]

llm = LocalLLM()
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)
qa.run("Summarize the main idea of the uploaded PDF.")



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12


KeyboardInterrupt: 

In [ ]:
!pip install langchain langchain-community faiss-cpu sentence-transformers transformers accelerate pymupdf gradio duckduckgo-search
from google.colab import files
uploaded = files.upload()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/1

Saving Kartikey Tiwari resume  4.pdf to Kartikey Tiwari resume  4.pdf


In [ ]:
!pip install -U langchain-community
!pip install pymupdf

from google.colab import files
uploaded = files.upload()

file_paths = list(uploaded.keys())  # list of uploaded files

from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("Kartikey Tiwari resume  4.pdf")
docs = loader.load()
print(f"✅ Loaded pages: {len(docs)}")

from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = splitter.split_documents(docs)
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = FAISS.from_documents(chunks, embedding_model)
retriever = db.as_retriever(search_kwargs={"k": 3})
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

from transformers import pipeline
from langchain.llms.base import LLM
from pydantic import Extra, Field
from typing import Optional, List, Any

class LocalLLM(LLM):
    pipeline: Any = Field(default=None)
    class Config:
        extra = Extra.allow

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.pipeline = pipeline("text2text-generation", model="google/flan-t5-large")

    @property
    def _llm_type(self) -> str:
        return "flan_t5"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        return self.pipeline(prompt[:1500], max_new_tokens=256)[0]["generated_text"]

llm = LocalLLM()
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)
result = qa.invoke("Summarize the main idea of the uploaded PDF.")
print(result)
result = qa.invoke("What is Kartikey Tiwari’s education background?")
print(result)
result = qa.invoke("List some projects mentioned in the resume.")
print(result)
result = qa.invoke("What skills are listed?")
print(result)

Saving Kartikey Tiwari resume  4.pdf.pdf to Kartikey Tiwari resume  4.pdf (1).pdf


ValueError: File path Kartikey Tiwari resume  4.pdf is not a valid file or url

In [2]:
# prompt: why my code give same answer to all different question

from google.colab import files

# Install necessary libraries
!pip install -q langchain langchain-community faiss-cpu sentence-transformers transformers accelerate pymupdf gradio duckduckgo-search

from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import pipeline
from langchain.llms.base import LLM
from pydantic import Extra, Field
from typing import Optional, List, Any
from langchain.chains import RetrievalQA

# Upload the file
uploaded = files.upload()

# Assuming only one file is uploaded and its name is "Kartikey Tiwari resume 4.pdf"
# If the file name is different, update the line below accordingly
file_name = list(uploaded.keys())[0] # Get the uploaded file name
if file_name not in uploaded:
    print(f"Error: File '{file_name}' not found in uploaded files.")
else:
    # Load and process the document
    loader = PyMuPDFLoader(file_name)
    docs = loader.load()
    print(f"✅ Loaded pages: {len(docs)}")

    splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=50)
    chunks = splitter.split_documents(docs)
    separators=["\n\n", "\n", ".", " ", ""]

    embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    db = FAISS.from_documents(chunks, embedding_model)
    retriever = db.as_retriever(search_kwargs={"k": 3})

    # Define and instantiate the LocalLLM
    class LocalLLM(LLM):
        pipeline: Any = Field(default=None)
        class Config:
            extra = Extra.allow

        def __init__(self, **kwargs):
            super().__init__(**kwargs)
            # Consider moving pipeline loading outside __init__ if memory is a concern
            # and you have multiple instances, though here it's a singleton.
            self.pipeline = pipeline("text2text-generation", model="google/flan-t5-large")

        @property
        def _llm_type(self) -> str:
            return "flan_t5"

        def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
            # FLAN-T5 has a max input length, truncating the prompt if necessary
            return self.pipeline(prompt[:1500], max_new_tokens=256)[0]["generated_text"]

    llm = LocalLLM()

    # Create the RetrievalQA chain
    qa = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        return_source_documents=True
    )

    # Ask multiple questions using the same qa object
    print("\n--- Asking Questions ---")

    question1 = "Summarize the main idea of the uploaded PDF."
    print(f"\nQ: {question1}")
    result1 = qa.invoke(question1)
    print(f"A: {result1['result']}")
    # print source documents if needed: print(f"Source Documents: {result1['source_documents']}")

    question2 = "What is Kartikey Tiwari’s education background?"
    print(f"\nQ: {question2}")
    result2 = qa.invoke(question2)
    print(f"A: {result2['result']}")

    question3 = "List some projects mentioned in the resume."
    print(f"\nQ: {question3}")
    result3 = qa.invoke(question3)
    print(f"A: {result3['result']}")

    question4 = "What skills are listed?"
    print(f"\nQ: {question4}")
    result4 = qa.invoke(question4)
    print(f"A: {result4['result']}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 862.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/1

Saving B.Tech EC Final Year.pdf to B.Tech EC Final Year.pdf
✅ Loaded pages: 18


/tmp/ipython-input-2-2799790154.py:36: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public mo

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu



--- Asking Questions ---

Q: Summarize the main idea of the uploaded PDF.
A: I Introduction: Overview of Image Processing, Application area of image processing, Digital Image Representation, Types of images, Digital Image Processing Operations, Fundamental steps in DIP, Overview of Digital Image Systems, Physical Aspect of Image Acquisition, biological Aspect of Image Acquisition, sampling & quantization, Digital Halftone Process, Image storage and File formats. 8 II Image Enhancement: Need for image enhancement, Image enhancement operations, Image enhancement in spatial domain, histogram based techniques, Spatial Filtering concepts, Image smoothing and sharpening spatial and frequency domain filters, homomorphic filtering. Image Restoration: Introduction to degradation, types of Image degradations, image degradation models, noise modeling, estimation of degradation functions, Image restoration in presence of noise only, periodic noise and band pass and band reject filtering, differen